In [1]:
from models import text_cnn

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}

In [4]:
num_filters = 3 
filter_sizes = [2, 2, 2] 
vocab_size = len(word_dict)
embedding_size = 2 
sequence_length = 3 
num_classes = 2 

In [5]:
model = text_cnn.TextCNN(
    num_filters, filter_sizes, vocab_size,
    embedding_size, sequence_length, num_classes
)
model

TextCNN(
  (W): Embedding(16, 2)
  (Weight): Linear(in_features=9, out_features=2, bias=False)
  (filter_list): ModuleList(
    (0-2): 3 x Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  )
  (batch_norm_list): ModuleList(
    (0-2): 3 x BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [13]:

model

TextCNN(
  (W): Embedding(16, 2)
  (Weight): Linear(in_features=9, out_features=2, bias=False)
  (filter_list): ModuleList(
    (0-2): 3 x Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  )
)

In [18]:
model = text_cnn.TextCNN(
    num_filters, filter_sizes, vocab_size,
    embedding_size, sequence_length, num_classes
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])
targets = torch.LongTensor([out for out in labels]) # To using Torch Softmax Loss function

# Training
for epoch in range(100):
    optimizer.zero_grad()
    output = model(inputs)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, targets)
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
 
    loss.backward()
    optimizer.step()

test_text = 'he hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = torch.LongTensor(tests)

# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

Epoch: 0001 cost = 0.668719
Epoch: 0002 cost = 0.667259
Epoch: 0003 cost = 0.665808
Epoch: 0004 cost = 0.664365
Epoch: 0005 cost = 0.662931
Epoch: 0006 cost = 0.661505
Epoch: 0007 cost = 0.660086
Epoch: 0008 cost = 0.658674
Epoch: 0009 cost = 0.657266
Epoch: 0010 cost = 0.655860
Epoch: 0011 cost = 0.654456
Epoch: 0012 cost = 0.653061
Epoch: 0013 cost = 0.651667
Epoch: 0014 cost = 0.650268
Epoch: 0015 cost = 0.648865
Epoch: 0016 cost = 0.647456
Epoch: 0017 cost = 0.646042
Epoch: 0018 cost = 0.644620
Epoch: 0019 cost = 0.643191
Epoch: 0020 cost = 0.641753
Epoch: 0021 cost = 0.640306
Epoch: 0022 cost = 0.638849
Epoch: 0023 cost = 0.637380
Epoch: 0024 cost = 0.635899
Epoch: 0025 cost = 0.634406
Epoch: 0026 cost = 0.632901
Epoch: 0027 cost = 0.631384
Epoch: 0028 cost = 0.629853
Epoch: 0029 cost = 0.628306
Epoch: 0030 cost = 0.626755
Epoch: 0031 cost = 0.625193
Epoch: 0032 cost = 0.623617
Epoch: 0033 cost = 0.622027
Epoch: 0034 cost = 0.620423
Epoch: 0035 cost = 0.618805
Epoch: 0036 cost = 0

In [8]:

model

TextCNN(
  (W): Embedding(16, 2)
  (Weight): Linear(in_features=9, out_features=2, bias=False)
  (filter_list): ModuleList(
    (0-2): 3 x Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  )
  (batch_norm_list): ModuleList(
    (0-2): 3 x BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [19]:
model = text_cnn.TextCNN(
    num_filters, filter_sizes, vocab_size,
    embedding_size, sequence_length, num_classes,is_batch_normalize=True
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])
targets = torch.LongTensor([out for out in labels]) # To using Torch Softmax Loss function

# Training
for epoch in range(100):
    optimizer.zero_grad()
    output = model(inputs)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, targets)
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
 
    loss.backward()
    optimizer.step()

test_text = 'he hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = torch.LongTensor(tests)

# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

Epoch: 0001 cost = 0.784973
Epoch: 0002 cost = 0.774597
Epoch: 0003 cost = 0.764506
Epoch: 0004 cost = 0.754701
Epoch: 0005 cost = 0.745181
Epoch: 0006 cost = 0.735939
Epoch: 0007 cost = 0.726968
Epoch: 0008 cost = 0.718814
Epoch: 0009 cost = 0.710768
Epoch: 0010 cost = 0.702837
Epoch: 0011 cost = 0.695028
Epoch: 0012 cost = 0.687339
Epoch: 0013 cost = 0.679760
Epoch: 0014 cost = 0.672283
Epoch: 0015 cost = 0.664895
Epoch: 0016 cost = 0.657579
Epoch: 0017 cost = 0.650321
Epoch: 0018 cost = 0.643109
Epoch: 0019 cost = 0.635927
Epoch: 0020 cost = 0.628943
Epoch: 0021 cost = 0.621962
Epoch: 0022 cost = 0.614938
Epoch: 0023 cost = 0.607859
Epoch: 0024 cost = 0.600757
Epoch: 0025 cost = 0.593628
Epoch: 0026 cost = 0.586453
Epoch: 0027 cost = 0.579147
Epoch: 0028 cost = 0.571704
Epoch: 0029 cost = 0.564150
Epoch: 0030 cost = 0.556454
Epoch: 0031 cost = 0.548588
Epoch: 0032 cost = 0.540539
Epoch: 0033 cost = 0.532375
Epoch: 0034 cost = 0.524589
Epoch: 0035 cost = 0.519719
Epoch: 0036 cost = 0